### Come gestire l'autenticazione: Keys o Resource Principal?

In [1]:
import oci
import torch

from oci.ai_vision import AIServiceVisionClient
from oci.ai_vision.models import AnalyzeImageDetails
from oci.ai_vision.models import ImageClassificationFeature
from oci.ai_vision.models import ImageObjectDetectionFeature
from oci.ai_vision.models import InlineImageDetails
from oci.ai_vision.models import ObjectStorageImageDetails

import os
import base64
import json
import cv2

### Auth Key (personali)

In [18]:
os.environ.get('PROJECT_COMPARTMENT_OCID')

'ocid1.compartment.oc1..aaaaaaaag2cpni5qj6li5ny6ehuahhepbpveopobooayqfeudqygdtfe6h3a'

In [3]:
# la configurazione è in questo file
!cat /home/datascience/.oci/config

[DEFAULT]
user=ocid1.user.oc1..aaaaaaaafurloh2ad5mdr6yukuupzpy72gcrad3gevaahzo7ws6t2yanvjka
fingerprint=67:6f:15:26:63:3f:6b:14:38:b9:ec:1e:e7:db:6f:4b
tenancy=ocid1.tenancy.oc1..aaaaaaaabu5fgingcjq3vc7djuwsdcutdxs4gsws6h4kfoldqpjuggxprgoa
region=eu-frankfurt-1
key_file=/home/datascience/.oci/luigi.saetta-05-16-10-29.pem


In [4]:
!ls -l /home/datascience/.oci

total 12
-rw-r--r--. 1 datascience users  323 Jun  9 08:46 config
-rw-r--r--. 1 datascience users 1730 Jun  9 08:45 luigi.saetta-05-16-10-29.pem
-rw-r--r--. 1 datascience users  458 Jun  9 08:45 luigi.saetta-05-16-10-29_public.pem


In [24]:
# legge le api key e la configurazione per invocare il Language Service
config = oci.config.from_file()

config

{'log_requests': False,
 'additional_user_agent': '',
 'pass_phrase': None,
 'user': 'ocid1.user.oc1..aaaaaaaafurloh2ad5mdr6yukuupzpy72gcrad3gevaahzo7ws6t2yanvjka',
 'fingerprint': '67:6f:15:26:63:3f:6b:14:38:b9:ec:1e:e7:db:6f:4b',
 'tenancy': 'ocid1.tenancy.oc1..aaaaaaaabu5fgingcjq3vc7djuwsdcutdxs4gsws6h4kfoldqpjuggxprgoa',
 'region': 'eu-frankfurt-1',
 'key_file': '/home/datascience/.oci/luigi.saetta-05-16-10-29.pem'}

In [25]:
# cosi va fatta l'invocazione
# Initialize client service_endpoint is optional if it's specified in config
# Vision Service endpoint
ENDPOINT = "https://vision.aiservice.eu-frankfurt-1.oci.oraclecloud.com"
# the custom trained model we're using for Object Detection
MODEL_ID = "ocid1.aivisionmodel.oc1.eu-frankfurt-1.amaaaaaangencdyarrrcfnwo2yly3goldt4zwvvcs2bkerzwfrbntli74sta"

ai_service_vision_client = AIServiceVisionClient(config=config, service_endpoint=ENDPOINT)

In [26]:
# get the list of AI Vision Models available in the chosen compartment
response = ai_service_vision_client.list_projects(compartment_id="ocid1.compartment.oc1..aaaaaaaahioj3gxhxpebugaocrtfbb25yramvyfs4ah2prmhnf2adovtmp6a")

response.data

{
  "items": [
    {
      "compartment_id": "ocid1.compartment.oc1..aaaaaaaahioj3gxhxpebugaocrtfbb25yramvyfs4ah2prmhnf2adovtmp6a",
      "defined_tags": {
        "default-tags": {
          "CreatedBy": "oracleidentitycloudservice/luigi.saetta@oracle.com"
        }
      },
      "display_name": "test_logo_images",
      "freeform_tags": {},
      "id": "ocid1.aivisionproject.oc1.eu-frankfurt-1.amaaaaaangencdyaiens6qiaablagzi5qogxrmxlvjhp42vevrtz4sy67fdq",
      "lifecycle_details": null,
      "lifecycle_state": "ACTIVE",
      "system_tags": {},
      "time_created": "2022-03-31T07:51:51.867000+00:00",
      "time_updated": "2022-03-31T07:51:59.366000+00:00"
    }
  ]
}

### Resource Principal (la NB session deve essere in un Dynamic Group aut. da policy)

In [29]:
# get the RP signer
rps = oci.auth.signers.get_resource_principals_signer()

In [30]:
ENDPOINT = "https://vision.aiservice.eu-frankfurt-1.oci.oraclecloud.com"
# the custom trained model we're using for Object Detection
MODEL_ID = "ocid1.aivisionmodel.oc1.eu-frankfurt-1.amaaaaaangencdyarrrcfnwo2yly3goldt4zwvvcs2bkerzwfrbntli74sta"

ai_service_vision_client = AIServiceVisionClient(config={}, service_endpoint=ENDPOINT, signer=rps)

In [31]:
# get the list of AI Vision Models available in the chosen compartment
response = ai_service_vision_client.list_projects(compartment_id="ocid1.compartment.oc1..aaaaaaaahioj3gxhxpebugaocrtfbb25yramvyfs4ah2prmhnf2adovtmp6a")

response.data

{
  "items": [
    {
      "compartment_id": "ocid1.compartment.oc1..aaaaaaaahioj3gxhxpebugaocrtfbb25yramvyfs4ah2prmhnf2adovtmp6a",
      "defined_tags": {
        "default-tags": {
          "CreatedBy": "oracleidentitycloudservice/luigi.saetta@oracle.com"
        }
      },
      "display_name": "test_logo_images",
      "freeform_tags": {},
      "id": "ocid1.aivisionproject.oc1.eu-frankfurt-1.amaaaaaangencdyaiens6qiaablagzi5qogxrmxlvjhp42vevrtz4sy67fdq",
      "lifecycle_details": null,
      "lifecycle_state": "ACTIVE",
      "system_tags": {},
      "time_created": "2022-03-31T07:51:51.867000+00:00",
      "time_updated": "2022-03-31T07:51:59.366000+00:00"
    }
  ]
}